### Initialization of objects

In [ ]:
import numpy as np
import pandas as pd
from numpy.random import default_rng
rng = default_rng()
import math
from math import log as log
from numpy.linalg import det as det
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
    
#############################################################################

#design matrix
cluster_1 = np.random.normal(-5, 0.5, (4, 20))
cluster_2 = np.random.normal(3, 0.09, (3, 20))
cluster_3 = np.random.normal(1, 0.9, (6, 20))
cluster_4 = np.random.normal(-2, 0.3, (2, 20))
discriminating_data = np.concatenate((cluster_1, cluster_2, cluster_3, cluster_4))
non_discriminating_data = np.random.normal(0, 1, (15, 980))
X = np.concatenate( (discriminating_data, non_discriminating_data), axis=1)
(n,p) = X.shape

#intialization
gamma = np.zeros(p)
gamma[rng.choice(np.arange(0, 1000), 10, replace = False)] = 1
c = np.arange(1, n+1)

#HYPERPARAMETERS

#Prior of the model on the discriminatory variables : a gaussian vector
mu_0 = np.array([np.median(X[:,j]) for j in range(p)]).reshape(p, 1)  #mean for the gaussian vector
h_1 = 1000   #multiplicatory coefficient for its variance-covariance Sigma
delta = 3   #mean of the Inverse Wishart prior for Sigma
kappa_1 = 0.0007   #variance multiplicator for the variance covariance matrix of the Inverse Wishart
Q_1 = kappa_1 * np.identity(p) #variance covariance matrix of the inverse Wishart
t = 5 #number of intermediate steps for launch state of latent vector c allocation update
gamma_total_iter = 20 # number of Metropolis updates of gamma vector of variable selection (authors say above 20 minimal improvement)
c_total_iter = 5 # number of updates of sample allocation vector c (authors of paper say above 5 minimal improvement)

#Prior of the model on non-discriminatory variables : a gaussian vector
h_0 = 100  #multiplicatory coefficient for its variance-covariance Omega
a=3        #first parameter of the Inverse Gamma prior on the constant variance sigma² of the non discriminatory (and assumed independent) elements
b = 0.2 #second parameter of the Inverse Gamma prior 

#Prior of gamma
omega = 10/p

alpha = 1

### Prior on gamma parameter FUN

In [ ]:
def prior_gamma(gamma):
    prior = 1
    for j in range(p):
        gamma_j = gamma[j]
        prior *= omega**gamma_j*(1-omega)**gamma_j
    return prior

### Log regularized versions of likelihood and a posteriori gamma distrib (FUNs) 

In [ ]:
def log_likelihood(X, gamma, c):
    L = (-n*p/2)*math.log(math.pi)
    K = len(np.unique(c))
    p_gamma = int(np.sum(gamma))
    gamma_indices = np.argwhere(gamma).transpose()[0]
    gammaC_indices = np.argwhere(gamma==0).transpose()[0]
    mu_0gamma = mu_0[gamma_indices]
    mu_0gammaC = mu_0[gammaC_indices]
    Q_1gamma = Q_1[gamma_indices, :][:, gamma_indices]   #pas sûr de cette définition


    for k in range(1, K+1):
        C_k = np.argwhere(c==k)
        n_k = len(C_k)
        x_kgamma = X[k-1, gamma_indices]

        H_kgamma = (h_1 * n_k + 1)**(-p_gamma/2)
        for j in range(1, p_gamma + 1):
            H_kgamma *= math.gamma( (n_k + delta + p_gamma -j)/2) / math.gamma( (delta + p_gamma -j)/2 )
        
        log_H_0gammaC = ( -(p - p_gamma)/2 )*log(h_0*n + 1) + ( a*(p-p_gamma) )*log(b)
        for j in range(1, p - p_gamma + 1):
            log_H_0gammaC += log(math.gamma(a+n/2)) - log(math.gamma(a))
            
        S_kgamma = n_k/(h_1*n_k +1)*(mu_0gamma - np.mean(x_kgamma))*np.transpose((mu_0gamma - np.mean(x_kgamma)))
        for i in C_k:
            x_igamma = X[i, gamma_indices]
            S_kgamma += (x_igamma - np.mean(x_kgamma))*np.transpose(x_igamma - np.mean(x_kgamma))
            
        log_S_0gammaC = 0
        for j in range(1, p - p_gamma + 1):
            sum_x = 0
            j_gammaC = gammaC_indices[j-1] #jth non discriminatory variable
            mu_0jgammaC = mu_0gammaC[j-1]
            for i in range(1, n+1):
                x_ijgammaC = X[i-1, j_gammaC]
                x_jgammaC = np.mean(X[:, j_gammaC])
                sum_x += (x_ijgammaC - np.mean(x_jgammaC))**2
            log_S_0gammaC += log(b + 1/2*(sum_x + n/(h_0*n+1)*(mu_0jgammaC - np.mean(x_jgammaC))**2))

        L += log(H_kgamma) + ((delta + p_gamma-1)/2)*log(abs(det(Q_1gamma))) - ((n_k + delta + p_gamma -1)/2)*log(abs(det(Q_1gamma + S_kgamma)))
        
    L += log_H_0gammaC - (a+n/2)*log_S_0gammaC
    return L


def log_conditional_aposteriori_gamma(X, gamma, c):
    return log_likelihood(X, gamma, c) + log(prior_gamma(gamma))

### gamma vector update (variable selection given c the latent vector of cluster alloc)

In [ ]:
def gamma_single_iter(gamma):
    """ Stochastic update Metropolis"""
    gamma_size = len(gamma)

    #stochastic update
    random = np.random.random()
    gamma_new = gamma.copy()
    if random < 1/2 and len(np.argwhere(gamma)) > 0 and len(np.argwhere(gamma)) < len(gamma):
        print("-> (gamma) we swap a 0 and a 1")
        #we swap a 0 and a 1
        gamma_zeros = np.argwhere(gamma==0)
        gamma_ones = np.argwhere(gamma)
        pick_zero = rng.choice(gamma_zeros)
        pick_one = rng.choice(gamma_ones)
        gamma_new[pick_zero] = 1 
        gamma_new[pick_one] = 0
        
    else: 
        print('-> (gamma) we pick an elem of gamma and change its value')
        random = np.random.random()
        gamma_zeros = np.argwhere(gamma==0)
        gamma_ones = np.argwhere(gamma)
        pick_zero = rng.choice(gamma_zeros)
        pick_one = rng.choice(gamma_ones)
        if (random > 1/2) & (len(gamma_ones) > 1):
            pick = pick_one
        else:
            pick = pick_zero
        
        # pick = rng.choice(gamma_size)
        print('-> (gamma) a ', int(gamma_new[pick]), ' becomes a ', int(abs(gamma_new[pick] - 1)))
        gamma_new[pick] = abs(gamma_new[pick] - 1)
    
    #apply metropolis probability of acceptance of the new array
    random = np.random.random()
    new_log_likelihood = log_conditional_aposteriori_gamma(X, gamma_new, c)
    print('-> (gamma) new L = ', new_log_likelihood)
    former_log_likelihood = log_conditional_aposteriori_gamma(X, gamma, c)
    print('-> (gamma) old L = ', former_log_likelihood)
    decision_threshold = min(1, math.exp(new_log_likelihood - former_log_likelihood))
    print('-> prob(gamma change) : ', decision_threshold)
    if random <= decision_threshold:
        print('-> gamma_new is retained')
        gamma = gamma_new
    else:
        print('-> gamma is unchanged')
    return gamma


### Sample alloc FUN

In [ ]:
def is_C_empty(n, c): # FUN for case splitting in samples alloc update (page 7/17)
    
    samples = [h for h in range(n)]
    i, l = rng.choice(samples, 2, replace = False)
    C = np.concatenate([np.argwhere(c == c[i]), np.argwhere(c == c[l])])
    C = C[C!=i]
    C = C[C!=l]
    answ = (len(C) == 0)
    print('-> TAG9 / is C empty : ', answ)
    return {'answer' : answ, 'Cval' : C, 'i,l' : [i,l]}


def Gibbs_scan(C_dict, c_launch, c, i, l, gamma_indices, mu_0gamma, p_gamma, Q_1gamma, merge_transition = False,):
    # EMILIEN
    log_transition_prob = 0
    
    for k in C_dict['Cval']:
        c_i = c_launch[i]
        if merge_transition:
            c_i = c[k]      #in that case, we'll want to compute the proba to go back to the original state
        c_l = c_launch[l]
        c_i_indices = np.argwhere(c_launch==c_i)
        c_l_indices = np.argwhere(c_launch==c_l)
        # required quantities for computation of pr(c_k|c_-k,...) :
        n_ci = sum(c_launch == c_launch[i])
        n_cl = sum(c_launch == c_launch[l])
        if c_launch[k] == c_launch[i]:
            n_kci = n_ci -1
        else:
            n_kci = n_ci
        if c_launch[k] == c_launch[l]:
            n_kcl = n_cl -1
        else:
            n_kcl = n_cl
        
        # building S-k,ci(gamma) and S-k,cl(gamma)
        if len(c_i_indices) > 0:
            x_cigamma_barre = np.mean(X[c_i_indices, gamma_indices], axis=0)
            S_cigamma = (1/(1+h_1))*(x_cigamma_barre - mu_0gamma)*np.transpose(x_cigamma_barre - mu_0gamma)
            S_k_cigamma = S_cigamma.copy()
            for index in c_i_indices:
                x_igamma = X[index, gamma_indices]
                to_add = (x_igamma - x_cigamma_barre)*np.transpose(x_igamma - x_cigamma_barre)
                S_cigamma += to_add
                if index != k:
                    S_k_cigamma += to_add
        if len(c_i_indices) == 0:
            S_cigamma = 0
            S_k_cigamma = 0
        
        if len(c_l_indices) > 0:
            x_clgamma_barre = np.mean(X[c_l_indices, gamma_indices], axis=0)
            S_clgamma = (1/(1+h_1))*(x_clgamma_barre - mu_0gamma)*np.transpose(x_clgamma_barre - mu_0gamma)
            S_k_clgamma = S_clgamma.copy()
            for index in c_l_indices:
                x_lgamma = X[index, gamma_indices]
                to_add = (x_lgamma - x_clgamma_barre)*np.transpose(x_lgamma - x_clgamma_barre)
                S_clgamma += to_add
                if index != k:
                    S_k_clgamma += to_add
        if len(c_l_indices) == 0:
            S_clgamma = 0
            S_k_clgamma = 0
        
        """# EMILIEN testing
        print('(-p_gamma/2)*log(math.pi) = ', (-p_gamma/2)*log(math.pi),
             '-(p_gamma/2)*log((h_1*n_ci+1)/(h_1*n_kci+1)) = ', -(p_gamma/2)*log((h_1*n_ci+1)/(h_1*n_kci+1)),
             'sum(...) = ', sum([log(math.gamma((n_ci+delta+p_gamma -j)/2)/math.gamma((n_kci+delta+p_gamma -j)/2)) for j in range(1, p_gamma+1)]),
             '-((n_ci + delta + p_gamma -1)/2)*log(abs(det(Q_1gamma + S_cigamma))) = ', -((n_ci + delta + p_gamma -1)/2)*log(abs(det(Q_1gamma + S_cigamma))),
             '((n_kci + delta + p_gamma -1)/2)*log(abs(det(Q_1gamma + S_k_cigamma))) = ', ((n_kci + delta + p_gamma -1)/2)*log(abs(det(Q_1gamma + S_k_cigamma))))
        print('det(Q_1gamma + S_cigamma) = ', det(Q_1gamma + S_cigamma), 'Q_1gamma = ', Q_1gamma, 'S_cigamma = ', S_cigamma)
        print('det(Q_1gamma + S_k_cigamma) = ', det(Q_1gamma + S_k_cigamma), 'Q_1gamma = ', Q_1gamma, 'S_k_cigamma = ', S_k_cigamma)
        """
        I_i = math.exp((-p_gamma/2)*log(math.pi) -(p_gamma/2)*log((h_1*n_ci+1)/(h_1*n_kci+1)) +
                        sum([log(math.gamma((n_ci+delta+p_gamma -j)/2)/math.gamma((n_kci+delta+p_gamma -j)/2)) for j in range(1, p_gamma+1)]) -
                        ((n_ci + delta + p_gamma -1)/2)*log(abs(det(Q_1gamma + S_cigamma))) +
                        ((n_kci + delta + p_gamma -1)/2)*log(abs(det(Q_1gamma + S_k_cigamma))))
        I_l = math.exp((-p_gamma/2)*log(math.pi) -(p_gamma/2)*log((h_1*n_cl+1)/(h_1*n_kcl+1)) +
                        sum([log(math.gamma((n_cl+delta+p_gamma -j)/2)/math.gamma((n_kcl+delta+p_gamma -j)/2)) for j in range(1, p_gamma+1)]) -
                        ((n_cl + delta + p_gamma -1)/2)*log(abs(det(Q_1gamma + S_clgamma))) +
                        ((n_kcl + delta + p_gamma -1)/2)*log(abs(det(Q_1gamma + S_k_clgamma))))
        
        # let's finally compute pr(c_k|c_-k,...) and assign sample k a label accordingly:
        pr_ci = n_kci*I_i/(n_kci*I_i + n_kcl*I_l)
        #print('IMP/ log_prob(ci) = ', log_pr_ci, ' ; pr_ci =', n_kci*I_i/(n_kci*I_i + n_kcl*I_l))
        #if np.isnan(log_pr_ci):
         #   return(print('log_pr_ci = ', log_pr_ci, 'n_kci = ', n_kci, ' I_i = ', I_i, 'n_kcl = ', n_kcl, ' I_l = ', I_l))
        
        if merge_transition:
            transition_prob *= pr_ci 

        else:
            random = rng.random()
            if min(1, pr_ci) >= random:    #regarder pr_k ou pr_cl
                print('-> TAG10 / min(1, pr_ci) >= random')
                c_launch[k] = c_launch[i]
                transition_prob *= pr_ci 
            else:
                print('-> TAG11 / min(1, pr_ci) < random')
                c_launch[k] = c_launch[l]
                transition_prob *= 1-pr_ci

        print('TAG12 / claunch, transition_prob = ', c_launch, transition_prob)
        
    return c_launch, transition_prob


def sample_alloc_update(n, c):
    
    C_dict = is_C_empty(n, c)
    
    if C_dict['answer']: # case 1 described at page 7
        i = C_dict['i,l'][0]
        l = C_dict['i,l'][1]
        print('entry c:', c)
        print(f'using i={i} and l={l} and c_i={c[i]} and c_l={c[l]}')

        # needed values for computation of a_csplit_c and a_cmerge_c
        p_gamma = int(np.sum(gamma))
        gamma_indices = np.argwhere(gamma).transpose()[0]
        mu_0gamma = mu_0[gamma_indices]
        Q_1gamma = Q_1[gamma_indices, :][:, gamma_indices]   #pas sûr de cette définition
        x_igamma = X[i, gamma_indices]
        x_lgamma = X[l, gamma_indices]
        S_igamma = (1/(1+h_1))*(x_igamma - mu_0gamma)*np.transpose(x_igamma - mu_0gamma)
        S_lgamma = (1/(1+h_1))*(x_lgamma - mu_0gamma)*np.transpose(x_lgamma - mu_0gamma)
        S_ilgamma = (1/(1+2*h_1))*((x_igamma - mu_0gamma)*np.transpose(x_igamma - mu_0gamma) + 
                                   (x_lgamma - mu_0gamma)*np.transpose(x_lgamma - mu_0gamma) + 
                                   h_1*(x_igamma - x_lgamma)*np.transpose(x_igamma - x_lgamma))
        vec = [(math.gamma((delta + p_gamma + 1 - j)/2)**2/(math.gamma((delta + p_gamma -j)/2)*math.gamma((delta + p_gamma +2 -j)/2))) for j in range(1, p_gamma +1)]
        prod_vec = np.prod(vec)
        
        if c[i] == c[l]:
            
            print('-> TAG13 / c_i == c_l true')
            c_split = c.copy()
            c_split[i] = max(c_split) + 1  
            a_csplit_c = min([1, math.exp(log(alpha) + log(prod_vec) + p_gamma*log((1+2*h_1)**(1/2)/(1+h_1)) +
                                          ((delta + p_gamma -1)/2)*log(abs(det(Q_1gamma))) + 
                                          ((delta + p_gamma +1)/2)*log(abs(det(Q_1gamma + S_ilgamma))) - 
                                          ((delta + p_gamma)/2)*log(abs(det(Q_1gamma + S_igamma)*det(Q_1gamma + S_lgamma))))])
            
            print('split proposal:', c_split)
            print('probability of acceptance:', a_csplit_c)
            acceptance_thres = rng.random()
            if a_csplit_c >= acceptance_thres:
                c = c_split
                print('split was accepted')
                print('c is now', c)

        else:
            
            print('-> TAG14 / c_i == c_l false')
            c_merge = c.copy()
            c_merge[i] = c_merge[l]
            a_cmerge_c = min([1, math.exp((-1)*log(alpha) + log(prod_vec) + p_gamma*log((1+2*h_1)**(1/2)/(1+h_1)) +
                                          ((delta + p_gamma -1)/2)*log(abs(det(Q_1gamma))) + 
                                          ((delta + p_gamma +1)/2)*log(abs(det(Q_1gamma + S_ilgamma))) - 
                                          ((delta + p_gamma)/2)*log(abs(det(Q_1gamma + S_igamma)*det(Q_1gamma + S_lgamma))))])
            
            print('merge proposal:', c_merge)
            print('probability of acceptance:', a_cmerge_c)
            acceptance_thres = rng.random()
            if a_cmerge_c >= acceptance_thres:
                c = c_merge
                print('merge was accepted')
                print('c is now', c)

    else: # case 2 described at page 8
        i = C_dict['i,l'][0]
        l = C_dict['i,l'][1]
        print('entry c:', c)
        print(f'using i={i} and l={l} and c_i={c[i]} and c_l={c[l]}')

        # needed values for computation of a_csplit_c and a_cmerge_c
        p_gamma = int(np.sum(gamma))
        gamma_indices = np.argwhere(gamma).transpose()[0]
        mu_0gamma = mu_0[gamma_indices]
        Q_1gamma = Q_1[gamma_indices, :][:, gamma_indices]   #pas sûr de cette définition
        x_igamma = X[i, gamma_indices]
        x_lgamma = X[l, gamma_indices]
        S_igamma = (1/(1+h_1))*(x_igamma - mu_0gamma)*np.transpose(x_igamma - mu_0gamma)
        S_lgamma = (1/(1+h_1))*(x_lgamma - mu_0gamma)*np.transpose(x_lgamma - mu_0gamma)
        S_ilgamma = (1/(1+2*h_1))*((x_igamma - mu_0gamma)*np.transpose(x_igamma - mu_0gamma) + 
                                   (x_lgamma - mu_0gamma)*np.transpose(x_lgamma - mu_0gamma) + 
                                   h_1*(x_igamma - x_lgamma)*np.transpose(x_igamma - x_lgamma))
        
        
        print('-> TAG15 / building launch state')
        c_launch = c.copy()
        

        if c[i] == c[l]:
            c_launch[i] = max(c) +1
            print('c_launch_i becomes', c_launch[i])
            print('c_launch_l is', c_launch[l])
        # else:
        #     c_launch[i] = c[l]
        
        # first scan with random assignments
        print('-> TAG16 / first scan with random assignments')
        for k in C_dict['Cval']:
            rand_draw = rng.random()
            if rand_draw < 1/2 :
                c_launch[k] = c_launch[i]
            else:
                c_launch[k] = c_launch[l]
        print('c_launch before scan:', c_launch)

        # required quantities for computation
        p_gamma = int(np.sum(gamma))
        gamma_indices = np.argwhere(gamma).transpose()[0]
        mu_0gamma = mu_0[gamma_indices]
        Q_1gamma = Q_1[gamma_indices, :][:, gamma_indices]
        
        # let's perform changes in vector c (those changes correspond to the launch state of the split-merge procedure page 8)
        for t_ in range(t):
            
            print('-> TAG17 / intermediate scans + changes on vector c for t_ in range t')
            c_launch, _ = Gibbs_scan(C_dict, c_launch, c, i, l, gamma_indices, mu_0gamma, p_gamma, Q_1gamma)
        
        
        
        # we now compute the proba a of accepting the new proposal c_split or c_merge :
        vec = [(math.gamma((delta + p_gamma + 1 - j)/2)**2/(math.gamma((delta + p_gamma -j)/2)*math.gamma((delta + p_gamma +2 -j)/2))) for j in range(1, p_gamma +1)]
        prod_vec = np.prod(vec)
        
        print('c_launch after Gibbs scan', c_launch)
        if c[i] == c[l]:
            
            c_split = c_launch.copy()
            c_split, transition_prob = Gibbs_scan(C_dict, c_split, c, i, l, gamma_indices, mu_0gamma, p_gamma, Q_1gamma)
            assert (c_split[l] == c[l])   #Check
            print('c_split proposal', c_split)
            # pas sûr qu'il faille calculer a_csplit_c comme ça... à voir en fonction de la proba de transition
            a_csplit_c = min([1, (1/transition_prob)*math.exp(log(alpha) + log(prod_vec) + p_gamma*log((1+2*h_1)**(1/2)/(1+h_1)) +
                                          ((delta + p_gamma -1)/2)*log(abs(det(Q_1gamma))) + 
                                          ((delta + p_gamma +1)/2)*log(abs(det(Q_1gamma + S_ilgamma))) - 
                                          ((delta + p_gamma)/2)*log(abs(det(Q_1gamma + S_igamma)*det(Q_1gamma + S_lgamma))))])
            acceptance_thres = rng.random()
            print("probability of acceptance of the split:", a_csplit_c)
            if a_csplit_c >= acceptance_thres:
                print('**-> TAG18 / a_csplit_c >= acceptance_thres')
                c = c_split
                
        else:
            
            c_merge = c.copy()
            c_merge[i] = c_merge[l] 
            c_merge[C_dict['Cval']] = [c_merge[l] for i in range(len(C_dict['Cval']))]
            _, transition_prob =  Gibbs_scan(C_dict, c_merge, c, i, l, gamma_indices, mu_0gamma, p_gamma, Q_1gamma, merge_transition=True)
            print('q quotient cmerge : ', math.exp(log_q_c_cmerge - log_q_cmerge_c))
            a_cmerge_c = min([1, transition_prob*math.exp((-1)*log(alpha) + log(prod_vec) + p_gamma*log((1+2*h_1)**(1/2)/(1+h_1)) +
                                          ((delta + p_gamma -1)/2)*log(abs(det(Q_1gamma))) + 
                                          ((delta + p_gamma +1)/2)*log(abs(det(Q_1gamma + S_ilgamma))) - 
                                          ((delta + p_gamma)/2)*log(abs(det(Q_1gamma + S_igamma)*det(Q_1gamma + S_lgamma))))])
            acceptance_thres = rng.random()
            print("probability of acceptance of the merge:", a_cmerge_c)
            if a_cmerge_c >= acceptance_thres:
                print('**-> TAG19 / a_cmerge_c >= acceptance_thres')
                c = c_merge
    
    #perform the final full Gibbs sampling scan (as described in p.9)
    
    for i in range(n):
        
        dict_probas  = {}
        for cl in np.unique(c):
            dict_probas[cl] = 0
        
        for l in range(n):
            if i != l:
                c_i = c[i]
                c_l = c[l]
                n_cl = len(c[c==c_l])    #number of elements that are assigned c_l
                n_icl = n_cl             
                if c_i == c_l:
                    n_icl -= 1           #number of elements that are not i and are assigned c_l
                
                #definition of S_clgamma and S_i_clgamma
                c_l_indices = np.argwhere(c==c_l)
                x_clgamma_barre = np.mean(X[c_l_indices, gamma_indices], axis=0)
                S_clgamma = (n_cl/(1+n_cl*h_1))*(x_clgamma_barre - mu_0gamma)*np.transpose(x_clgamma_barre - mu_0gamma)
                S_i_clgamma = S_clgamma.copy()
                for index in c_l_indices:
                    x_lgamma = X[index, gamma_indices]
                    to_add = (x_lgamma - x_clgamma_barre)*np.transpose(x_lgamma - x_clgamma_barre)
                    S_clgamma += to_add
                    if index != i:
                        S_i_clgamma += to_add
                
                #compute the probability (up to a normalization constant) of c_i = c_l
                # prob = math.pi**( (-n*p_gamma)/2) * n_icl/(n-1+alpha) * ( (h_1*n_cl + 1)/(h_1*n_icl + 1) )**(-p_gamma/2) * abs(det(Q_1gamma + S_clgamma))**(- (n_cl + delta + p_gamma -1)/2 ) * abs(det(Q_1gamma + S_i_clgamma))**((n_icl + delta + p_gamma -1)/2)
                log_prob = math.log(math.pi)*( (-n*p_gamma)/2) + math.log(n_icl/(n-1+alpha)) + math.log(( (h_1*n_cl + 1)/(h_1*n_icl + 1) ))*(-p_gamma/2) + math.log(abs(det(Q_1gamma + S_clgamma)))*(- (n_cl + delta + p_gamma -1)/2 ) + math.log(abs(det(Q_1gamma + S_i_clgamma)))*((n_icl + delta + p_gamma -1)/2)
                for j in range(p_gamma):
                    log_prob += math.log(math.gamma( (n_cl + delta + p_gamma - j)/2)/math.gamma( (n_icl + delta + p_gamma -j)/2 ))
                
                # (Emilien) modifié 04122022_1853 : différents samples peuvent avoir le même cluster selon moi 
                # d'où le += ci-dessous (si pas d'accord dis moi hein)
                dict_probas[c_l] += math.exp(log_prob)
        
        #compute the probability (up to a constant) of c_i =/= c_l for all l: probability to keep c_i ?
        x_igamma = X[i, gamma_indices]
        S_igamma = 1/(h_1+1)*(x_igamma - mu_0gamma)*np.transpose(x_igamma - mu_0gamma)
        log_prob = math.log(math.pi)*( (-n*p_gamma)/2) + math.log(alpha/(n-1+alpha)) + math.log(h_1 + 1)*(-p_gamma/2) + math.log(det(Q_1gamma))*( (delta+p_gamma-1)/2 ) + math.log(abs(det(Q_1gamma + S_igamma)))*( -(delta+p_gamma)/2 )    #a besoin du abs pour pas avoir de problèmes de signes
        for j in range(p_gamma):
            log_prob += math.log(math.gamma( (1 + delta + p_gamma - j)/2)/math.gamma( (delta + p_gamma -j)/2 ))
        # pareil on sait pas peut-être que le cluster c_i a déjà eu des probas d'être le cluster de i dans les boucles sur l
        dict_probas[c_i] += math.exp(log_prob)

        normalization_constant = sum(dict_probas.values())
        for key, value in dict_probas.items():
            dict_probas[key] = value/normalization_constant
        c[i] = rng.choice(list(dict_probas.keys()), p=list(dict_probas.values()))
        
    print('c after final gibbs scan', c)
    return c
                   

### Chain call FUN

In [ ]:
def chain(gamma, n, c, loop_num):
    
    for m in range(loop_num):
        print(f"**RUNNING {m}th iteration")
        for iter in range(gamma_total_iter):
            print(f"{iter} iteration of gamma")
            gamma = gamma_single_iter(gamma)
            printmd("**Num. of significant variables**")
            print(sum(gamma == 1))
            
        
        for iter in range(c_total_iter): 
            print(f"{iter} iteration of c")
            c = sample_alloc_update(n, c)
            printmd("**Clusters alloc**")
    
    return c, gamma

In [ ]:
c, gamma = chain(gamma, n, c, 10)

### Tentative alternative

In [ ]:
def c_update(n, c):
    
    samples = [h for h in range(n)]
    i, l = rng.choice(samples, 2, replace = False)

    # needed values for computation of a_csplit_c and a_cmerge_c
    p_gamma = int(np.sum(gamma))
    gamma_indices = np.argwhere(gamma).transpose()[0]
    mu_0gamma = mu_0[gamma_indices]
    Q_1gamma = Q_1[gamma_indices, :][:, gamma_indices]   #pas sûr de cette définition
    x_igamma = X[i, gamma_indices]
    x_lgamma = X[l, gamma_indices]
    S_igamma = (1/(1+h_1))*(x_igamma - mu_0gamma)*np.transpose(x_igamma - mu_0gamma)
    S_lgamma = (1/(1+h_1))*(x_lgamma - mu_0gamma)*np.transpose(x_lgamma - mu_0gamma)
    S_ilgamma = (1/(1+2*h_1))*((x_igamma - mu_0gamma)*np.transpose(x_igamma - mu_0gamma) + 
                               (x_lgamma - mu_0gamma)*np.transpose(x_lgamma - mu_0gamma) + 
                               h_1*(x_igamma - x_lgamma)*np.transpose(x_igamma - x_lgamma))
    vec = [(math.gamma((delta + p_gamma + 1 - j)/2)**2/(math.gamma((delta + p_gamma -j)/2)*math.gamma((delta + p_gamma +2 -j)/2))) for j in range(1, p_gamma +1)]
    prod_vec = np.prod(vec)

    if c[i] == c[l]:

        print('-> c_i == c_l')
        c_split = c.copy()
        c_split[i] = max(c_split) + 1  
        a_csplit_c = min([1, math.exp(log(alpha) + log(prod_vec) + p_gamma*log((1+2*h_1)**(1/2)/(1+h_1)) +
                                      ((delta + p_gamma -1)/2)*log(abs(det(Q_1gamma))) + 
                                      ((delta + p_gamma +1)/2)*log(abs(det(Q_1gamma + S_ilgamma))) - 
                                      ((delta + p_gamma)/2)*log(abs(det(Q_1gamma + S_igamma)*det(Q_1gamma + S_lgamma))))])

        #print('split proposal:', c_split)
        #print('probability of acceptance:', a_csplit_c)
        acceptance_thres = rng.random()
        if a_csplit_c >= acceptance_thres:
            c = c_split
            print('-> split was accepted, c is now ', c)
        
        return(c)

    else:

        print('-> c_i != c_l false')
        c_merge = c.copy()
        c_merge[i] = c_merge[l]
        a_cmerge_c = min([1, math.exp((-1)*log(alpha) + log(prod_vec) + p_gamma*log((1+2*h_1)**(1/2)/(1+h_1)) +
                                      ((delta + p_gamma -1)/2)*log(abs(det(Q_1gamma))) + 
                                      ((delta + p_gamma +1)/2)*log(abs(det(Q_1gamma + S_ilgamma))) - 
                                      ((delta + p_gamma)/2)*log(abs(det(Q_1gamma + S_igamma)*det(Q_1gamma + S_lgamma))))])

        #print('merge proposal:', c_merge)
        #print('probability of acceptance:', a_cmerge_c)
        acceptance_thres = rng.random()
        if a_cmerge_c >= acceptance_thres:
            c = c_merge
            print('-> merge was accepted, c is now ', c)
            
        return(c)

In [ ]:
def reindexing(c):
    reindexing = {}
    new_label = 1
    for label in np.unique(c):
        reindexing[label] = new_label
        new_label += 1
    for i in range(len(c)):
        c[i] = reindexing[c[i]]
    
    return c


def chain_alter(gamma, n, c, loop_num):
    
    c_storage = []
    gamma_storage = []
    
    for m in range(loop_num):
        print(f"**RUNNING {m}th iteration")
        for iter in range(gamma_total_iter):
            print(f"{iter} iteration of gamma")
            gamma = gamma_single_iter(gamma)
            gamma_storage.append(gamma)
            printmd("**Num. of significant variables**")
            print(sum(gamma == 1))
            
        
        for iter in range(c_total_iter): 
            print(f"{iter} iteration of c")
            c = reindexing(c_update(n, c))
            c_storage.append(c)
            printmd("**Clusters alloc**")
    
    return c, gamma, c_storage, gamma_storage

In [ ]:
#c = np.arange(1, n+1)
init_index = [i for i in range(int(n/2+1))]
c[init_index] = 1

gamma_total_iter = 20 # number of Metropolis updates of gamma vector of variable selection (authors say above 20 minimal improvement)
c_total_iter = 200 # number of updates of sample allocation vector c (authors of paper say above 5 minimal improvement)

In [ ]:
c, gamma, c_storage, gamma_storage = chain_alter(gamma, n, c, 10000)

Edits 11/12
- avoir avec mêmes probas la proposition de changer un 0 et 1 et un 1 en 0
- x_igamma = X[i, gamma_indices] car i est dans range(n) et c[i] correspond à la i-eme ligne de X
- x_cigamma = X[c[i]-1, gamma_indices] devient x_cigamma_barre = np.mean(X[c_i_indices, gamma_indices], axis=0) car c'est la moyenne des élements du cluster c_i 
- pr_k = n_kck*I_k/(n_kci*I_i + n_kcl*I_l) devient pr_ci = n_kci*I_i/(n_kci*I_i + n_kcl*I_l) car le gibbs sampling doit permettre de choisir entre ci_launch et cj_launch pour tous les k dans C_k et on a bien d'ailleurs pr_ci + pr_cl = 1
- transition prob remplacé par 1/transition_prob dans a_csplit car q(c^split|c) est au dénominateur
- Ajout du gibbs sampling final manquant dans la construction de c_split (Case 2.b.ii). Pour éviter copier coller création d'une fonction gibbs_sampling à part

Commit 2: 
- redéfinition des transitions prob à l'aide de l'article original mais semble obtenir des probas trop hautes pour les merge et trop faibles pour les split




Changements du 12/12_Emilien

* debugging des pr_ci = nan : solution :correction du calcul de S_cigamma quand ci_indexes est vide
* tentative d'ajouter la proba de transition au dénominateur mais on obtient une proba d'acceptation toujours de 1 pour les merge comme les split (j'ai remis en l'état du 11/12 avant de commit)
* tentative de simplifier la maj de c avec min(1, le rapport des vraisemblances) comme proba d'acceptation : chain_alter